<a href="https://colab.research.google.com/github/LeeYuuuan/Graph-Kernel-Isomorphic-/blob/main/Sum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install grakel
import grakel
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import networkx.algorithms.isomorphism as iso
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score



import networkx.algorithms.isomorphism as iso
from sklearn.model_selection import cross_val_score

from sklearn.metrics import roc_curve, auc
from sklearn import metrics, svm
from sklearn.preprocessing import label_binarize
from sklearn.metrics import classification_report 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn.pipeline import make_pipeline
from sklearn.multiclass import OneVsRestClassifier
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.multiclass import OneVsRestClassifier


def draw_graphs(Gra1,Gra2):
    pos = nx.spring_layout(Gra1)
    pos1 = nx.spring_layout(Gra2) 
    plt.subplot(121)
    nx.draw(Gra1, pos, with_labels=True, node_color='pink', node_size=500)
    plt.subplot(122)
    nx.draw(Gra2, pos1, with_labels=True, node_size=500)
    plt.show()

def GetGraphData(dataset_name):
    dataset = grakel.datasets.fetch_dataset(dataset_name, verbose=True, data_home=None, download_if_missing=True, with_classes=True, produce_labels_nodes=False, prefer_attr_nodes=False, prefer_attr_edges=False, as_graphs=False)
    G = dataset.data
    y = dataset.target
    return G, y

def GetGraph(an_old_graph):
    """
    This function input a graph from /grakel dataset/
    Then converted to a type [class]Graph (networkx)
    """
    Graph = nx.MultiGraph()
    Graph.add_edges_from(list(an_old_graph[0]))

    if len(an_old_graph[1]) != 0:
      for node in an_old_graph[1]:
        Graph.add_node(node, attr=an_old_graph[1][node])
    if len(an_old_graph[2]) !=0:
      for edge in an_old_graph[2]:
        Graph.add_edge(edge[0],edge[1], attr=an_old_graph[2][edge])
  
    return Graph

def GetGraphwithoutLabel(an_old_graph):
    """
    This function input a graph from /grakel dataset/
    Then converted to a type [class]Graph (networkx) 
    NOTE: without labels.
    """
    Graph = nx.MultiGraph()
    Graph.add_edges_from(list(an_old_graph[0]))

    return Graph

def GetAllGraph(graph_dataset):
    """
    Get all Graph with type of [class]Graph
    return a list [Graph1, Graph2, ...]
    """
    Graphs = []
    for graph_data in graph_dataset:
      Graphs.append(GetGraph(graph_data))
    
    return Graphs

def GetAllGraphwithoutLables(graph_dataset):
    """
    Get all Graph with type of [class]Graph
    return a list [Graph1, Graph2, ...]
    """
    Graphs = []
    for graph_data in graph_dataset:
      Graphs.append(GetGraphwithoutLabel(graph_data))
    
    return Graphs

def GetSumCount(Graphs):
    """
    Get the number of the graphs in the dataset.
    """
    return len(Graphs)

def GetAverageNodes(Graphs):
    """
    Get the average number of nodes for a certain dataset.
    """
    sum = 0
    for Graph in Graphs:
        sum += len(Graph.nodes())
    
    return sum/len(Graphs)


def GetAverageEdges(Graphs):
    """
    Get the average number of edges for a certain dataset.
    """
    sum = 0
    for Graph in Graphs:
        sum += len(Graph.edges())
    
    return sum/len(Graphs)/2

def GetMaxandMinNode(Graphs):
    """
    Get the max and min number of nodes in Graphs.
    """
    max_number = - 1
    min_number = 999999
    for Graph in Graphs:
        if len(Graph.nodes()) < min_number:
            min_number = len(Graph.nodes())
        
        if len(Graph.nodes()) > max_number:
            max_number = len(Graph.nodes())
    return [max_number, min_number]


def GetLabelCount(dataset_name):
    """
    Get the number of each labels.
    """
    G,y = GetGraphData(dataset_name)
   
    values, counts = np.unique(y, return_counts=True)
    for i in range(len(values)):
        print("label: ",values[i], "count:", counts[i])

def GetSummary(name):
    """
    Get the summary statisic information for a certain Dataset.

    or Get for a list of Datasets.
    """
    if type(name) == str :
        G,y = GetGraphData(name)
        Graphs = GetAllGraph(G)
        df = pd.DataFrame(columns=["Name","Graphs","Class","Average Nodes", "Average Edges", "Max number of nodes", "Min number of nodes"])
        temp_df = {"Name": name, 
                   "Graphs": GetSumCount(Graphs), 
                   "Class": len(set(y)), 
                   "Average Nodes": GetAverageNodes(Graphs),
                   "Average Edges": GetAverageEdges(Graphs),
                   "Max number of nodes": GetMaxandMinNode(Graphs)[0],
                   "Min number of nodes": GetMaxandMinNode(Graphs)[1]
                   }
        df = df.append([temp_df], ignore_index=True)
        # df.loc[0] = [name, GetSumCount(Graphs), len(set(y)), GetAverageNodes(Graphs)]
    elif type(name) == list:
        df = pd.DataFrame(columns=["Name","Graphs","Class","Average Nodes", "Average Edges", "Max number of nodes", "Min number of nodes"])
        for dataset in name:
            G,y = GetGraphData(dataset)
            Graphs = GetAllGraph(G)
            
            temp_df = {"Name": dataset, 
                   "Graphs": GetSumCount(Graphs), 
                   "Class": len(set(y)), 
                   "Average Nodes" : GetAverageNodes(Graphs),
                   "Average Edges" : GetAverageEdges(Graphs),
                   "Max number of nodes": GetMaxandMinNode(Graphs)[0],
                   "Min number of nodes": GetMaxandMinNode(Graphs)[1]
                   }
            df = df.append(temp_df, ignore_index=True)
    return df

def IsIsomorphicDefault(G1,G2,is_node_labeled,is_edge_labeled):
  """
  Input two Graphs G1, G2,
  Input True/False if Node/edge is labeled or not.

  return True/False if two Graphs is isomorphic or not.
  """
  if is_node_labeled:
    nm = iso.numerical_node_match("attr",0)
    if is_edge_labeled:
      em = iso.numerical_edge_match("attr",-1)
      GM = iso.GraphMatcher(G1, G2, node_match=nm, edge_match=em)
    else:
      GM = iso.GraphMatcher(G1, G2, node_match=nm)
  else:
    if is_edge_labeled:
      em = iso.numerical_edge_match("attr",-1)
      GM = iso.GraphMatcher(G1, G2, edge_match=em)
    else:
      GM = iso.GraphMatcher(G1, G2)
  
  return np.float64(GM.is_isomorphic())


def CreateFeatureMatrix(graphs,is_node_labeled,is_edge_labeled):
    feature_matrix = np.zeros([len(graphs),len(graphs)])
    for i in range(len(graphs)):
      for j in range(len(graphs)):
        feature_matrix[i,j] = IsIsomorphicDefault(graphs[i],graphs[j],is_node_labeled,is_edge_labeled)
   
  
    return feature_matrix

def GetIsomorphicFeatureMatrix(dataset_name,is_node_labeled,is_edge_labeled):
    G, y = GetGraphData(dataset_name)
    all_graph = GetAllGraph(G)
    return CreateFeatureMatrix(all_graph,is_node_labeled,is_edge_labeled), y
  

def DrawIsomorphicMatrix(dataset_name, is_node_labeled, is_edge_labeled):
    sumIso = 0
    fm, y = GetIsomorphicFeatureMatrix(dataset_name,is_node_labeled,is_edge_labeled)        
    plt.matshow(fm)
    print("there are ",(np.sum(fm) - fm.shape[0])/2, "pairs of graphs isomorphiced" )


def GetNodeDistribution(dataset_name):
    """
    draw the node distribution histogram.
    """
    G,y = GetGraphData(dataset_name)
    Graphs = GetAllGraph(G)
    node_count = []

    for graph in Graphs:
        node_count.append(len(graph.nodes()))
    

    label_bins = len(np.unique(y))

    max_nodes = GetMaxandMinNode(Graphs)[0]
    print("VISUALIZATIONS:\nMax nodes in consideration: {}".format(max_nodes))
    plt.figure(figsize=(12, 5))
    plt.subplot(121)
    
    plt.hist(node_count, bins=len(np.unique(node_count)))
    plt.xlabel('Number of Nodes in Graph', fontsize=12)
    plt.ylabel('Count', fontsize=12)

    plt.subplot(122)
    plt.hist2d(y, node_count, bins=[label_bins, 20])
    plt.xlabel(r'Graph label', fontsize=12)
    plt.ylabel(r'Graph size (number of nodes)', fontsize=12)
    plt.colorbar()
    plt.show()
    
    print("Correlation between graph size (number of nodes) and labels: %.2f" % np.corrcoef(y, node_count)[0,1])


def CreateFeatureMatrix_wl_test(vector_all):
  feature_matrix = np.zeros([len(vector_all),len(vector_all[0])])
  for i in range(len(vector_all)):
      feature_matrix[i,:] = vector_all[i]
      #feature_matrix[i,j] = IsIsomorphicDefault(graphs[i],graphs[j],is_node_labeled,is_edge_labeled)
   
  return feature_matrix

def Classification(dataset_name, kernel_name):
    G, y = GetGraphData(dataset_name)
    G_train, G_test, y_train, y_test = train_test_split(G, y, test_size=0.1, random_state=42)

    C_grid = (10. ** np.arange(-4,6,1) / len(G)).tolist()
    if kernel_name == "weisfeiler_lehman":

        gk = grakel.GraphKernel(kernel=[{"name": "weisfeiler_lehman", "n_iter": 5}, "subtree_wl"] ,Nystroem=20)
        K_train = gk.fit_transform(G_train)
        K_test = gk.transform(G_test)

        clf = SVC(kernel='linear')
        clf.fit(K_train, y_train)
        SVC(kernel='linear')
        C_grid = (10. ** np.arange(-4,6,1) / len(G)).tolist()
        scores = cross_val_score(clf, K_train, y_train, cv=5)
        y_pred = clf.predict(K_test)
        print(str(round(accuracy_score(y_test, y_pred)*100, 2)), "%")
      
    
    if kernel_name == "ShortestPath":
        estimator = make_pipeline(
        grakel.kernels.ShortestPath(normalize=True),
        GridSearchCV(SVC(kernel='precomputed'), dict(C=C_grid),
                    scoring='accuracy', cv=10))
        
        n_folds = 10
        cvp = cross_val_predict(estimator, G, y, cv=n_folds)
        acc = accuracy_score(y, cvp)
        print("Accuracy:", str(round(acc*100, 2)) + "%")
        print(classification_report(y,cvp))

        # sns.heatmap(cm,cmap = sns.color_palette("Blues"),annot = True



def IsomorphicClassification(dataset_name,label_n):
    
    

    f_m, y = GetIsomorphicFeatureMatrix(dataset_name,label_n,False)

    G_train, G_test, y_train, y_test = train_test_split(f_m, y, test_size=0.1, random_state=42)
    clf = SVC(kernel='linear')
    clf.fit(G_train, y_train)
    SVC(kernel='linear')
    y_pred = clf.predict(G_test)
    print(classification_report(y_test, y_pred))



def IsomorphicClassification(dataset_name,label_n):
    
    

    f_m, y = GetIsomorphicFeatureMatrix(dataset_name,label_n,False)

    G_train, G_test, y_train, y_test = train_test_split(f_m, y, test_size=0.1, random_state=42)
    clf = SVC(kernel='linear')
    clf.fit(G_train, y_train)
    SVC(kernel='linear')
    y_pred = clf.predict(G_test)
    print(classification_report(y_test, y_pred))



